In [22]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



                                              0.0/11.1 MB ? eta -:--:--
                                              0.3/11.1 MB 8.6 MB/s eta 0:00:02
     --                                       0.8/11.1 MB 10.1 MB/s eta 0:00:02
     ----                                     1.4/11.1 MB 9.8 MB/s eta 0:00:01
     ------                                   1.8/11.1 MB 10.6 MB/s eta 0:00:01
     -------                                  2.1/11.1 MB 10.1 MB/s eta 0:00:01
     --------                                 2.3/11.1 MB 8.6 MB/s eta 0:00:02
     ---------                                2.8/11.1 MB 8.9 MB/s eta 0:00:01
     ----------                               3.0/11.1 MB 8.7 MB/s eta 0:00:01
     ------------                             3.4/11.1 MB 8.4 MB/s eta 0:00:01
     ------------                             3.6/11.1 MB 8.5 MB/s eta 0:00:01
     ---------------                          4.3/11.1 MB 8.5 MB/s eta 0:00:01
     -----------------                        4.8/11.1 

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [95]:
df = pd.read_parquet("dataset_pisa.parquet")

In [107]:
# ============================
# Variables para el modelo
# ============================

# Subcompetencias de matemática
subcompetencias = [
    'PV1MATH', 'PV2MATH', 'PV3MATH', 'PV4MATH', 'PV5MATH',
    'PV6MATH', 'PV7MATH', 'PV8MATH', 'PV9MATH', 'PV10MATH'
]

# Factores directos
otros_factores_directos = ['AGE', 'TFGender']

# Variables de actividad académica
variables_actividad = [
    'REPEAT', 'MISSSC', 'SKIPPING', 'TARDYSD',
    'EXERPRAC', 'STUDYHMW', 'WORKPAY', 'WORKHOME'
]
# ============================
# Subcompetencias de Matemática reales (según PISA)
# ============================


# Variables compuestas
BELONG_inputs = ['ST034Q01TA', 'ST034Q02TA', 'ST034Q03TA', 'ST034Q04TA', 'ST034Q05TA', 'ST034Q06TA']
FAMSUP_inputs = ['ST300Q01JA', 'ST300Q02JA', 'ST300Q03JA', 'ST300Q04JA', 'ST300Q05JA',
                 'ST300Q06JA', 'ST300Q07JA', 'ST300Q08JA', 'ST300Q09JA', 'ST300Q10JA']
TEACHSUP_inputs = ['ST270Q01JA', 'ST270Q02JA', 'ST270Q03JA', 'ST270Q04JA']
BULLIED_inputs = ['ST038Q03NA', 'ST038Q04NA', 'ST038Q05NA', 'ST038Q06NA', 'ST038Q07NA', 'ST038Q08NA']
HOME_resources_inputs = ['ST250Q01JA', 'ST250Q02JA', 'ST250Q03JA', 'ST250Q04JA', 'ST250Q05JA',
                         'ST251Q01JA', 'ST251Q02JA', 'ST251Q03JA', 'ST251Q04JA', 'ST255Q01JA']

# Agrupación total de columnas a usar
columnas_utiles = (
    subcompetencias +
    otros_factores_directos +
    variables_actividad +
    BELONG_inputs +
    FAMSUP_inputs +
    TEACHSUP_inputs +
    BULLIED_inputs +
    HOME_resources_inputs
)

df_modelo = df[columnas_utiles].copy()

# ============================
# Limpieza de valores
# ============================

# Reemplazar valores nulos
df_modelo.replace(["", "NA", "NaN", "9999"], np.nan, inplace=True)

# ----------------------------
# Mapeo de variables categóricas
# ----------------------------

# Variables categóricas a transformar
mapeos_generales = {
    # Escalas de frecuencia
    "Never or almost never": 1,
    "About once or twice a year": 2,
    "About once or twice a month": 3,
    "About once or twice a week": 4,
    "Every day or almost every day": 5,

    # Escalas escolares
    "Some lessons": 2,
    "Most lessons": 3,
    "Every lesson": 4,

    # Acuerdo / desacuerdo
    "Strongly disagree": 1,
    "Disagree": 2,
    "Agree": 3,
    "Strongly agree": 4,

    # Recursos en el hogar
    "None": 0,
    "One": 1,
    "Two": 2,
    "Three or more": 3,
    
    "No": 0,
    "Yes": 1,

    # Escalas numéricas declaradas como texto
    "0": 0,
    "1": 1,
    "2": 2,
    "3": 3,
    "4": 4,
    "5": 5,
    "6": 6,
    "7": 7,
    "8": 8,
    "9": 9,
    "10 or more": 10,
    'There are no books.': 0,
    '1-10 books': 1,
    '11-25 books': 2,
    '26-100 books': 3,
    '101-200 books': 4,
    '201-500 books': 5,
    'More than 500 books': 6
}

# Columnas a aplicar mapeo general
columnas_a_mapear = (
    BELONG_inputs +
    FAMSUP_inputs +
    TEACHSUP_inputs +
    BULLIED_inputs +
    HOME_resources_inputs
)

for col in columnas_a_mapear:
    df_modelo[col] = df_modelo[col].map(mapeos_generales)

# Variables de actividad
actividad_mapeo = {
    "Never repeated": 0,
    "Repeated at lease once": 1,
    "Never missed school for >3 months": 0,
    "Missed school for >3 months at least once": 1,
    "No classes skipped in the prior two weeks": 0,
    "At lease one class skipped in the prior two weeks": 1,
    "On-time (not late in the prior two weeks)": 0,
    "Occasionally late (late 1 to 2 times in the prior two weeks)": 1,
    "Frequently late (late 3 or more times in the prior two weeks)": 2,
}


df_modelo["REPEAT"] = df_modelo["REPEAT"].map(actividad_mapeo)
df_modelo["MISSSC"] = df_modelo["MISSSC"].map(actividad_mapeo)
df_modelo["SKIPPING"] = df_modelo["SKIPPING"].map(actividad_mapeo)
df_modelo["TARDYSD"] = df_modelo["TARDYSD"].map(actividad_mapeo)

# Mapeo de frecuencia de actividades numéricas (de texto a número)
def map_actividad_frecuencia(valor):
    if isinstance(valor, str):
        if "No" in valor:
            return 0
        elif "1 time" in valor:
            return 1
        elif "2 times" in valor:
            return 2
        elif "3 times" in valor:
            return 3
        elif "4 times" in valor:
            return 4
        elif "5 times" in valor:
            return 5
        elif "6 times" in valor:
            return 6
        elif "7 times" in valor:
            return 7
        elif "8 times" in valor:
            return 8
        elif "9 times" in valor:
            return 9
        elif "10 or more" in valor:
            return 10
    return pd.to_numeric(valor, errors='coerce')

for col in ["EXERPRAC", "STUDYHMW", "WORKPAY", "WORKHOME"]:
    df_modelo[col] = df_modelo[col].apply(map_actividad_frecuencia)

# Mapeo de género si viene en texto
genero_map = {"Female": 0, "Male": 1}
df_modelo["TFGender"] = df_modelo["TFGender"].map(genero_map)


# Definimos cada grupo de PVs por subcompetencia
subcompetencias_pisa = {
    'MATH_CANTIDAD':        [f'PV{i}MCQN' for i in range(1, 11)],
    'MATH_CAMBIO_REL':      [f'PV{i}MCCR' for i in range(1, 11)],
    'MATH_ESPACIO_FORMA':   [f'PV{i}MCSS' for i in range(1, 11)],
    'MATH_DATOS_INCERT':    [f'PV{i}MCUD' for i in range(1, 11)],
    'MATH_FORMULACION':     [f'PV{i}MPFS' for i in range(1, 11)],
    'MATH_PROCEDIMIENTOS':  [f'PV{i}MPEM' for i in range(1, 11)],
    'MATH_INTERPRETACION':  [f'PV{i}MPIN' for i in range(1, 11)],
    'MATH_RAZONAMIENTO':    [f'PV{i}MPRE' for i in range(1, 11)],
}

# Calcular el promedio de los 10 valores por subcompetencia
for nombre_sub, columnas in subcompetencias_pisa.items():
    df_modelo[nombre_sub] = df[columnas].mean(axis=1)

# Eliminar columnas originales si ya no se usarán
columnas_a_quitar = [col for cols in subcompetencias_pisa.values() for col in cols]
df_modelo.drop(columns=columnas_a_quitar, inplace=True, errors='ignore')

# Crear una columna con el promedio general de subcompetencias
df_modelo['MATH_PROMEDIO'] = df_modelo[list(subcompetencias_pisa.keys())].mean(axis=1)

# Etiqueta binaria: ¿Logra o no la competencia? (ajustar umbral si es necesario)
df_modelo['MATH_LOGRO'] = (df_modelo['MATH_PROMEDIO'] >= 480).astype(int)

# Guardar dataset preprocesado si lo deseas
df_modelo.to_csv("pisa_modelo_preparado.csv", index=False)

In [108]:
df_modelo

,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,PV9MATH,PV10MATH,...,MATH_CANTIDAD,MATH_CAMBIO_REL,MATH_ESPACIO_FORMA,MATH_DATOS_INCERT,MATH_FORMULACION,MATH_PROCEDIMIENTOS,MATH_INTERPRETACION,MATH_RAZONAMIENTO,MATH_PROMEDIO,MATH_LOGRO
0,345.399,324.989,372.074,354.885,364.118,357.311,332.899,273.631,338.097,306.864,...,362.7859,315.0251,390.0648,349.1953,351.4022,342.3986,370.9045,355.1504,354.615850,0
1,392.984,398.520,354.017,431.459,388.498,335.722,367.239,428.286,391.804,387.961,...,378.3739,380.1831,401.3640,339.6969,384.8071,381.5699,396.3179,376.8208,379.891700,0
2,298.479,326.762,273.155,290.603,274.588,332.643,312.488,365.739,296.818,337.437,...,325.7199,305.2620,315.2557,251.5995,308.6171,320.0725,277.2500,266.2319,296.251075,0
3,334.534,291.940,265.874,302.434,213.076,271.113,291.715,334.264,281.487,242.324,...,302.6336,287.0586,281.9532,268.3772,214.0079,294.0478,290.6676,295.3500,279.261988,0
4,414.582,394.416,396.977,398.126,403.720,444.169,405.652,418.055,437.709,408.744,...,407.1886,404.4650,425.7730,382.8660,441.4261,402.1376,400.4027,406.9772,408.904525,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6963,411.570,377.703,348.770,380.878,372.930,320.592,394.324,380.480,369.485,377.377,...,352.3284,380.0228,383.5738,412.3215,397.6733,333.8210,400.4342,322.1915,372.795813,0
6964,325.992,326.672,356.414,322.564,384.702,360.172,380.549,328.399,373.243,329.527,...,336.1657,322.4276,369.2863,359.7526,375.2751,337.2989,362.2892,344.9346,350.928750,0
6965,446.236,438.534,463.147,437.231,439.159,391.715,469.553,434.306,473.552,431.527,...,445.7462,435.3555,395.9593,486.5884,436.7459,414.8984,423.6410,417.8161,432.093850,0
6966,401.329,349.522,446.524,425.930,422.287,426.663,409.283,384.803,389.028,359.680,...,399.5480,377.5185,391.7641,402.5334,424.1356,420.2140,389.2453,353.4036,394.795312,0


In [109]:
# Verificar cuáles de las columnas tienen solo nulos (no se pueden imputar)
for col in columnas_categoricas:
    if df_modelo[col].isnull().all():
        print(f"❌ La columna '{col}' tiene solo valores nulos.")


In [110]:
from sklearn.impute import SimpleImputer
import pandas as pd

# 1. Subcompetencias + edad (media)
imputador_media = SimpleImputer(strategy='mean')
df_modelo[subcompetencias + ['AGE']] = imputador_media.fit_transform(df_modelo[subcompetencias + ['AGE']])

# 2. Columnas categóricas (moda)
imputador_moda = SimpleImputer(strategy='most_frequent')

# Filtrar solo columnas que no están completamente vacías
columnas_validas = [col for col in columnas_categoricas if not df_modelo[col].isnull().all()]

# Aplicar imputación
imputados = imputador_moda.fit_transform(df_modelo[columnas_validas])
df_imputados = pd.DataFrame(imputados, columns=columnas_validas, index=df_modelo.index)

# Reemplazar columnas en el dataframe
df_modelo[columnas_validas] = df_imputados


In [111]:
df_modelo.head()

,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,PV9MATH,PV10MATH,...,MATH_CANTIDAD,MATH_CAMBIO_REL,MATH_ESPACIO_FORMA,MATH_DATOS_INCERT,MATH_FORMULACION,MATH_PROCEDIMIENTOS,MATH_INTERPRETACION,MATH_RAZONAMIENTO,MATH_PROMEDIO,MATH_LOGRO
0,345.399,324.989,372.074,354.885,364.118,357.311,332.899,273.631,338.097,306.864,...,362.7859,315.0251,390.0648,349.1953,351.4022,342.3986,370.9045,355.1504,354.615850,0
1,392.984,398.520,354.017,431.459,388.498,335.722,367.239,428.286,391.804,387.961,...,378.3739,380.1831,401.3640,339.6969,384.8071,381.5699,396.3179,376.8208,379.891700,0
2,298.479,326.762,273.155,290.603,274.588,332.643,312.488,365.739,296.818,337.437,...,325.7199,305.2620,315.2557,251.5995,308.6171,320.0725,277.2500,266.2319,296.251075,0
3,334.534,291.940,265.874,302.434,213.076,271.113,291.715,334.264,281.487,242.324,...,302.6336,287.0586,281.9532,268.3772,214.0079,294.0478,290.6676,295.3500,279.261988,0
4,414.582,394.416,396.977,398.126,403.720,444.169,405.652,418.055,437.709,408.744,...,407.1886,404.4650,425.7730,382.8660,441.4261,402.1376,400.4027,406.9772,408.904525,0


In [112]:
df_modelo.to_csv("preprocessing_pisa_data.csv", index=False)

In [106]:
df_modelo.head()

,PV1MATH,PV2MATH,PV3MATH,PV4MATH,PV5MATH,PV6MATH,PV7MATH,PV8MATH,PV9MATH,PV10MATH,...,ST250Q01JA,ST250Q02JA,ST250Q03JA,ST250Q04JA,ST250Q05JA,ST251Q01JA,ST251Q02JA,ST251Q03JA,ST251Q04JA,ST255Q01JA
0,345.399,324.989,372.074,354.885,364.118,357.311,332.899,273.631,338.097,306.864,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0
1,392.984,398.520,354.017,431.459,388.498,335.722,367.239,428.286,391.804,387.961,...,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,298.479,326.762,273.155,290.603,274.588,332.643,312.488,365.739,296.818,337.437,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0
3,334.534,291.940,265.874,302.434,213.076,271.113,291.715,334.264,281.487,242.324,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,2.0
4,414.582,394.416,396.977,398.126,403.720,444.169,405.652,418.055,437.709,408.744,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0


In [ ]:
# Entrenamiento RNN

In [92]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, accuracy_score

ImportError: Traceback (most recent call last):
  File "c:\Users\Pipo\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Error en una rutina de inicialización de biblioteca de vínculos dinámicos (DLL).


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.

In [85]:
df = pd.read_csv("preprocessing_pisa_data.csv")

In [86]:

subcompetencias = [
    'PV1MATH_target', 'PV2MATH_target', 'PV3MATH_target', 'PV4MATH_target', 'PV5MATH_target',
    'PV6MATH_target', 'PV7MATH_target', 'PV8MATH_target', 'PV9MATH_target', 'PV10MATH_target'
]

# Features (sin targets)
X = df.drop(columns=subcompetencias)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))  # Para RNN: (samples, time_steps, features)

# Paso 3: Entrenamiento por cada subcompetencia
resultados = {}

for target in subcompetencias:
    print(f"\n🧠 Entrenando modelo para: {target}")
    
    y = df[target]
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
    
    # Paso 4: Definir modelo RNN
    model = Sequential()
    model.add(SimpleRNN(64, activation='tanh', input_shape=(1, X_train.shape[2]), return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))  # Para clasificación binaria

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # Paso 5: Entrenar
    history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2, verbose=0)

    # Paso 6: Evaluar
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    acc = accuracy_score(y_test, y_pred)
    print(f"🔍 Accuracy: {acc:.4f}")
    print(classification_report(y_test, y_pred))

    resultados[target] = {
        "accuracy": acc,
        "reporte": classification_report(y_test, y_pred, output_dict=True)
    }

KeyError: "['PV1MATH_target', 'PV2MATH_target', 'PV3MATH_target', 'PV4MATH_target', 'PV5MATH_target', 'PV6MATH_target', 'PV7MATH_target', 'PV8MATH_target', 'PV9MATH_target', 'PV10MATH_target'] not found in axis"